In this first step, we will load the HiP-CT scan data from the specified input path (`path_input`) into a NumPy array. This array will serve as the basis for subsequent segmentation and analysis steps. The data type will be converted to `uint32` to ensure compatibility with downstream processing functions.

In [ ]:
from hipct_segmenter import load_hip_ct_scan_to_np
import numpy as np

# Load the HiP-CT scan as a NumPy array and convert its data type to uint32
# For faster processing use a folder with few handpicked slices

path_input = 'path '

array = load_hip_ct_scan_to_np(path_input).astype(np.uint32)


Loading 1274 JP2 slices...


JP2 Loading: 100%|██████████| 1274/1274 [00:16<00:00, 74.97slice/s]


Loaded Dataset 160.08um_A129_lung_complete-organ_pag-0.08_0.20_jp2_ (JP2)


Here we create an instance of the HiPCTSegmenter Class

In [2]:
from hipct_segmenter import HiPCTSegmenter

segmenter = HiPCTSegmenter()

In the next step, we apply the `tune_threshold` method from the `HiPCTSegmenter` class to a single slice of the loaded HiP-CT scan data. This function generates a binary mask by thresholding the image, which helps to separate regions of interest (such as tissue) from the background.

- **Parameter `threshold`**: This is the intensity threshold value. Pixels with values above this threshold are considered part of the organ, while those below are excluded. Adjusting this value changes the sensitivity of the segmentation.
- **Parameter `scan_sclice`**: This selects a fixed slice from the 3D scan array for thresholding. Choosing different slices allows you to tune the threshold for representative sections of the data.
- **Parameter `enhance_contrast=True`**: This enables contrast for plotting to improve visualization.



In [ ]:
threshold_mask = segmenter.tune_threshold(threshold=19000, scan_slice=array[600], enhance_contrast=True)


In this step, we use the `tune_ring` method from the `HiPCTSegmenter` class to refine the initial threshold mask. This function is designed to remove the container the organ is stored in

- **Parameter `orig_mask`**: The binary mask generated from the previous thresholding step, representing the initial segmentation.
- **Parameter `radius`**: The radius of the ring to be extracted from the mask. Adjusting this value changes the size of the region of interest.
- **Parameter `width`**: The thickness of the ring. Increasing this value will include a broader region around the specified radius.
- **Parameter `offset`**: The (x, y) offset to shift the center of the ring, allowing for fine-tuning of the region's position.


In [ ]:
ring_mask = segmenter.tune_ring(orig_mask=threshold_mask, radius=510, width=80, offset=(0, 0)) 

In this step, we apply the `tune_surrounded_cut` method from the `HiPCTSegmenter` class to further refine the mask obtained from the ring extraction. The algorithm works by examining each pixel in the mask and checking for the presence of organ pixels in preset ranges along the four main (horizontal and vertical) directions from the starting pixel. If each direction contains at least one organ pixel, the starting pixel is classified as a organ pixel. If not, the algorithm checks the four diagonal directions using the same principle. If neither condition is met, the pixel remains as background. This process is repeated for every pixel in the scan and iterated two additional times to ensure thorough filling of the mask.

- **Parameter `orig_mask`**: The binary mask from the previous ring extraction step.
- **Parameter `merge_size`**: Minimum size for merging connected regions; larger values merge more extensive regions.
- **Parameter `diag_size`** and **`cross_size`**: Sizes of the diagonal lines and cross lines around the invesitgated pixel that are investigated for mask pixels.
- **Parameter `iterations`**: Number of times the process is repeated to ensure complete filling of the mask.

This step helps to fill out the mask by ensuring that only regions fully surrounded by organ pixels are retained, improving the accuracy and completeness of the organ segmentation.

In [ ]:
surrounded_mask = segmenter.tune_surrounded_cut(orig_mask=ring_mask, merge_size=62, diag_size=7, cross_size=7, iterations=3)

In this step, we use the `tune_neighborhood_cut` method from the `HiPCTSegmenter` class to further refine the mask. This function examines the local neighborhood of each pixel in the mask and retains pixels that meet a specified threshold of neighboring mask pixels.

- **Parameter `surrounded_mask`**: The binary mask from the previous surrounded cut step.
- **Parameter `size`**: The size of the neighborhood window (in pixels) to consider around each pixel.
- **Parameter `threshold`**: The minimum fraction of neighboring pixels that must be part of the mask for the central pixel to be retained.

This operation helps to remove small isolated regions and smooth the mask, resulting in a cleaner segmentation.

In [ ]:
neighborhood_mask = segmenter.tune_neighborhood_cut(surrounded_mask, size=10, threshold=0.35)

In this step, we use the `fill_boxes` method from the `HiPCTSegmenter` class to further process the mask. This function fills in box-shaped regions within the mask, which can help to close gaps or holes that may remain after previous segmentation steps.

- **Parameter `neighborhood_mask`**: The binary mask from the previous neighborhood cut step.

This operation ensures that the segmented mask is more contiguous and suitable for subsequent analysis or visualization.

In [ ]:
filled_boxes_mask = segmenter.fill_boxes(neighborhood_mask)

In this step, we visualize the segmentation results by plotting a selection of slices from the original HiP-CT scan data. The `plot_segmented_slices` method from the `HiPCTSegmenter` class is used for this purpose.

- **Parameter `array`**: The original 3D scan data loaded earlier.
- **Parameter `num_slices`**: The number of slices to display from the scan.
- **Parameter `n_cpus`**: The number of CPU cores to use for parallel processing during the segmentation.
- **Parameter `figsize`**: The size of the figure for each plotted slice.
- **Parameter `save_path`**: If provided, the plots will be saved to this path; if `None`, the plots are displayed inline.

This visualization step helps to qualitatively assess the segmentation performance across multiple slices of the scan. Before applying it to the complete scan

In [ ]:
segmenter.plot_segmented_slices(array, num_slices=20, n_cpus=4, figsize=(5, 20), save_path=None)

In the next step, we export the current segmentation parameters to a YAML configuration file using the `export_params_to_yaml` method of the `HiPCTSegmenter` class. 
Now the segmenation_script can be run to apply the parameters to the whole scan. This is done as running a pyhton_script on high performance clusters is easier than rannung an notebook. 

In [ ]:
path_config = "./config_seg.yaml"
segmenter.export_params_to_yaml(path_config)